In [1]:
import json
import csv
import pandas as pd
import numpy as np
import geopandas as gdp
import datetime
import keplergl

In [2]:
filepath_trips = 'insert path here'

# Data cleaning

In [3]:
#Opening dataset and converting to DataFrame
with open(filepath_trips) as f_obj:
    scooter_trips = json.load(f_obj)

In [4]:
scooter_trips_df = pd.DataFrame.from_dict(scooter_trips, orient='index')

In [5]:
scooter_trips_df['Trip Distance'] = pd.to_numeric(scooter_trips_df['Trip Distance'])
scooter_trips_df['Trip Duration'] = pd.to_numeric(scooter_trips_df['Trip Duration'])

In [6]:
#First we fill in blank cells with NaN-values
scooter_trips_df = scooter_trips_df.replace(r'^\s*$', np.nan, regex=True)

In [7]:
# Dropping rows with missing values of time, distance, trip lenght or coordinates
trips_df = scooter_trips_df.dropna(axis=0, subset=['Start Time', 'End Time', 'Trip Distance', 'Trip Duration',
                                                          'Start Centroid Latitude','Start Centroid Longitude',
                                                          'End Centroid Latitude','End Centroid Longitude',])

In [8]:
#Dropping rows where distance is less than 100 meters or time is less than 60 seconds
trips_df = trips_df.drop(trips_df[(trips_df['Trip Distance'] < 100) | (trips_df['Trip Duration'] < 60)].index)

In [9]:
# Changing types in DataFrame
trips_df['Accuracy'] = pd.to_numeric(trips_df['Accuracy'])
trips_df['Start Census Tract'] = pd.to_numeric(trips_df['Start Census Tract'])
trips_df['End Census Tract'] = pd.to_numeric(trips_df['End Census Tract'])
trips_df['Start Community Area Number'] = pd.to_numeric(trips_df['Start Community Area Number'])
trips_df['End Community Area Number'] = pd.to_numeric(trips_df['End Community Area Number'])
trips_df['Start Centroid Latitude'] = pd.to_numeric(trips_df['Start Centroid Latitude'])
trips_df['Start Centroid Longitude'] = pd.to_numeric(trips_df['Start Centroid Longitude'])
trips_df['End Centroid Latitude'] = pd.to_numeric(trips_df['End Centroid Latitude'])
trips_df['End Centroid Longitude'] = pd.to_numeric(trips_df['End Centroid Longitude'])

# Preparing for maps

In [10]:
all_trips = trips_df.loc[:, ['Trip ID','Start Time','Start Centroid Latitude', 'Start Centroid Longitude',
                                    'End Centroid Latitude','End Centroid Longitude',
                             'Start Community Area Number',]].copy()

In [11]:
#Renaming column
all_trips = all_trips.rename(columns={'Start Community Area Number': 'area_number'})

In [12]:
# Converting datetime to str values
all_trips['Start Time'] = all_trips['Start Time'].astype(str)

In [13]:
# Summarizing number of trips based on area code
area_dict = all_trips["area_number"].value_counts()

## Creating second layer

### Creating second choropleth layer

In [14]:
# Adding path to shapely file containing area codes
path_area_code ='insert path here'

In [15]:
boundaries_community = gdp.read_file(path_area_code)

In [16]:
boundaries_community.head(n=3)

,area,area_num_1,area_numbe,comarea,comarea_id,community,perimeter,shape_area,shape_len,geometry
0,0.0,35,35,0.0,0.0,DOUGLAS,0.0,4.600462e+07,31027.054510,"POLYGON ((-87.60914 41.84469, -87.60915 41.844..."
1,0.0,36,36,0.0,0.0,OAKLAND,0.0,1.691396e+07,19565.506153,"POLYGON ((-87.59215 41.81693, -87.59231 41.816..."
2,0.0,37,37,0.0,0.0,FULLER PARK,0.0,1.991670e+07,25339.089750,"POLYGON ((-87.62880 41.80189, -87.62879 41.801..."


In [17]:
boundaries = boundaries_community.loc[:, ['area_numbe','community','geometry',]].copy()

In [18]:
#Converting column values from area_numbe to numeric value
boundaries['area_numbe'] = pd.to_numeric(boundaries['area_numbe'])

In [19]:
# Creating column containing total number of trips per community area
boundaries["area_value"] = boundaries["area_numbe"].map(area_dict)

In [20]:
boundaries.head()

,area_numbe,community,geometry,area_value
0,35,DOUGLAS,"POLYGON ((-87.60914 41.84469, -87.60915 41.844...",NaN
1,36,OAKLAND,"POLYGON ((-87.59215 41.81693, -87.59231 41.816...",NaN
2,37,FULLER PARK,"POLYGON ((-87.62880 41.80189, -87.62879 41.801...",NaN
3,38,GRAND BOULEVARD,"POLYGON ((-87.60671 41.81681, -87.60670 41.816...",NaN
4,39,KENWOOD,"POLYGON ((-87.59215 41.81693, -87.59215 41.816...",NaN


In [21]:
# Replacing NaN´s with 0´s
boundaries['area_value'] = boundaries['area_value'].fillna(0)

# Creating Kepler.gl-map

In [22]:
#Creating base map
map1 = keplergl.KeplerGl(height=600, widht=800)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [23]:
# Adding layer
map1.add_data(data=boundaries, name="Community Boundaries")

In [24]:
#Adding second layer
map1.add_data(data=all_trips, name="Trip Data")

In [25]:
#Display map
map1

KeplerGl(data={'Community Boundaries': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17…

In [26]:
#Show configuration
map1.config

{'version': 'v1',
 'config': {'visState': {'filters': [],
   'layers': [{'id': 'g4s9l2a',
     'type': 'geojson',
     'config': {'dataId': 'Community Boundaries',
      'label': 'Community Boundaries',
      'color': [18, 147, 154],
      'columns': {'geojson': 'geometry'},
      'isVisible': True,
      'visConfig': {'opacity': 0.8,
       'strokeOpacity': 0.8,
       'thickness': 0.5,
       'strokeColor': [221, 178, 124],
       'colorRange': {'name': 'Uber Viz Sequential 4',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#E6FAFA',
         '#C1E5E6',
         '#9DD0D4',
         '#75BBC1',
         '#4BA7AF',
         '#00939C']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radius': 10,
       'sizeRange': [0, 10],
       'radiusRange': [0, 50],


In [27]:
#Saving configuration
my_config= map1.config
print(my_config)

{'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'g4s9l2a', 'type': 'geojson', 'config': {'dataId': 'Community Boundaries', 'label': 'Community Boundaries', 'color': [18, 147, 154], 'columns': {'geojson': 'geometry'}, 'isVisible': True, 'visConfig': {'opacity': 0.8, 'strokeOpacity': 0.8, 'thickness': 0.5, 'strokeColor': [221, 178, 124], 'colorRange': {'name': 'Uber Viz Sequential 4', 'type': 'sequential', 'category': 'Uber', 'colors': ['#E6FAFA', '#C1E5E6', '#9DD0D4', '#75BBC1', '#4BA7AF', '#00939C']}, 'strokeColorRange': {'name': 'Global Warming', 'type': 'sequential', 'category': 'Uber', 'colors': ['#5A1846', '#900C3F', '#C70039', '#E3611C', '#F1920E', '#FFC300']}, 'radius': 10, 'sizeRange': [0, 10], 'radiusRange': [0, 50], 'heightRange': [0, 500], 'elevationScale': 5, 'stroked': True, 'filled': True, 'enable3d': False, 'wireframe': False}, 'hidden': False, 'textLabel': [{'field': None, 'color': [255, 255, 255], 'size': 18, 'offset': [0, 0], 'anchor': 'start

In [28]:
#Saving to html
map1.save_to_html(config=my_config, file_name="chicago_electric_scooters.html")

Map saved to chicago_electric_scooters.html!
